In [14]:
filename = "attn_256_256" # "attn_256_128"

In [15]:
import pickle
with open(filename+'.pkl',"rb") as f:
    configs,cost_configs, cycles_list = pickle.load(f)

configs[0].Br

32

In [ ]:
from cost_model.arch.a100 import A100
from cost_model.arch.RTX4090 import RTX4090
import torch

arch = RTX4090() if torch.cuda.get_device_capability(0) == (8,9) else A100() if torch.cuda.get_device_capability(0) == (8,0) else None

In [16]:
import pandas as pd
data = {
    "fuse_type": [config.fuse_type for config in configs],
    "tile1": [(config.Br, config.Bc, config.BlockKSmem) for config in configs],
    "tile2": [(config.Br, config.D, config.BlockKSmem2) for config in configs],
    "warps": [config.Nthreads//32 for config in configs],
    "warps_layout1": [(config.Nthreads//32//config.warps_mma1_n,config.warps_mma1_n) for config in configs],
    "warps_layout2": [(config.Nthreads//32//config.warps_mma_n,config.warps_mma_n) for config in configs],
    "unrollLastIter": [config.unrollLastIter for config in configs],
    "ncu_cycles": cycles_list,
    "peak_time": [cycles/arch.max_freq*1000 for cycles in cycles_list],
}
df = pd.DataFrame(data)
df.to_csv(filename+".csv")

In [17]:
from cost_model.example_op.flash_attention_reg import calculate_flash_attention_reg_resource_utilization
from cost_model.example_op.flash_attention_shared import calculate_flash_attention_shared_resource_utilization


cost_model_list = []
for i,cost_config in enumerate(cost_configs):
    if configs[i].fuse_type == "register":
        cost_model_list.append(calculate_flash_attention_reg_resource_utilization(*cost_config, arch=arch))
    elif configs[i].fuse_type == "shared":
        cost_model_list.append(calculate_flash_attention_shared_resource_utilization(*cost_config, arch=arch))

In [18]:
data["predict time(ms)"] = [pred*1000 for pred in  cost_model_list]
data["pred/peak"] = [pred/peak for pred,peak in zip(data["predict time(ms)"],data["peak_time"])]
df = pd.DataFrame(data)
df.to_csv(filename+"_pred.csv")